# Requirments

In [ ]:
!pip install pandas
!pip install absl-py
!pip install seaborn
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install matplotlib

In [115]:
import asyncio
import datetime
import shutil
import tempfile
import warnings
from pathlib import Path

from absl import app, flags, logging

import pandas as pd
import seaborn as sns
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, LeakyReLU, Dropout


# Main

In [106]:
data_dir = Path(r'C:\Users\luke1\Desktop\projects\T-brain-SinoPac-AIGO\data\training_data.csv')
data_columns = {
    'ID': 'ID',
    '縣市': 'City',
    '鄉鎮市區': 'District',
    '路名': 'Street_Name',
    '土地面積': 'Land_Area',
    '使用分區': 'Zoning',
    '移轉層次': 'Transfer_Level',
    '總樓層數': 'Total_Floors',
    '主要用途': 'Primary_Use',
    '主要建材': 'Primary_Construction_Material',
    '建物型態': 'Building_Type',
    '屋齡': 'Age_of_Building',
    '建物面積': 'Building_Area',
    '車位面積': 'Parking_Area',
    '車位個數': 'Number_of_Parking_Spaces',
    '橫坐標': 'Longitude',
    '縱坐標': 'Latitude',
    '備註': 'Remarks',
    '主建物面積': 'Main_Building_Area',
    '陽台面積': 'Balcony_Area',
    '附屬建物面積': 'Auxiliary_Building_Area',
    '單價': 'Unit_Price'
    }


In [116]:
# async def main_async():
#     print('hi')

def main():
    # asyncio.run(main_async())

    df = pd.read_csv(data_dir)
    
    df = df.rename(columns=data_columns)

    df=df.drop(["ID", "Longitude","Latitude", "Street_Name", "Remarks"], axis=1)
    df['Address'] = df['City'] + df['District']
    df = df.drop(columns=['City', 'District'])
    # print(df.head)

    # One hot encoding example
    # df = pd.get_dummies(df, columns=['Primary_Use'])

    # Label encoding
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df['Zoning'] = label_encoder.fit_transform(df['Zoning'])
    Zoning_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Primary_Use'] = label_encoder.fit_transform(df['Primary_Use'])
    Primary_Use_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Primary_Construction_Material'] = label_encoder.fit_transform(df['Primary_Construction_Material'])
    Primary_Construction_Material_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Building_Type'] = label_encoder.fit_transform(df['Building_Type'])
    Building_Type_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    df['Address'] = label_encoder.fit_transform(df['Address'])
    Address_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    # print(Zoning_mapping)


    # Feature Scaling
    scaler = StandardScaler()
    df['Land_Area'] = scaler.fit_transform(df['Land_Area'].values.reshape(-1, 1))
    df['Age_of_Building'] = scaler.fit_transform(df['Age_of_Building'].values.reshape(-1, 1))
    df['Building_Area'] = scaler.fit_transform(df['Building_Area'].values.reshape(-1, 1))
    df['Parking_Area'] = scaler.fit_transform(df['Parking_Area'].values.reshape(-1, 1))
    df['Main_Building_Area'] = scaler.fit_transform(df['Main_Building_Area'].values.reshape(-1, 1))
    df['Balcony_Area'] = scaler.fit_transform(df['Balcony_Area'].values.reshape(-1, 1))
    df['Auxiliary_Building_Area'] = scaler.fit_transform(df['Auxiliary_Building_Area'].values.reshape(-1, 1))
    df['Unit_Price'] = scaler.fit_transform(df['Unit_Price'].values.reshape(-1, 1))

    # to get each pairplot
    # pairplot(df)

    # shuffle
    seed=5811
    np.random.seed(seed)
    dataset = df.values
    np.random.shuffle(dataset)

    # seperate dataset
    X = df.drop(columns=['Unit_Price'])
    y = df['Unit_Price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

    # method
    # train(X_train, X_test, y_train, y_test)
    # call(X_test, y_test)

    # Assuming X_train, Y_train, X_test, Y_test are your training and testing data
    model = train_and_evaluate(X_train, y_train, X_test, y_test)

    pickle.dump(model, open(r'output\output.pickle', "wb"))

    # print(Y_pred)
if __name__ == "__main__":
    main()

MSE_train:  0.43861284101460113
MAE_train:  0.46650549311090966
MSE_test:  0.4218248580717224
MAE_test:  0.4565343122079575


c:\Users\luke1\Desktop\projects\T-brain-SinoPac-AIGO\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(


In [111]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def build_model():
    model = MLPRegressor(hidden_layer_sizes=(32,), activation="relu", solver="adam", 
                         max_iter=80, random_state=42)
    return model

def train_and_evaluate(X_train, Y_train, X_test, Y_test):
    model = build_model()
    model.fit(X_train, Y_train)  # Fit the model with training data
    
    Y_pred_train = model.predict(X_train)
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mae_train = mean_absolute_error(Y_train, Y_pred_train)
    
    print("MSE_train: ", mse_train)
    print("MAE_train: ", mae_train)
    
    Y_pred_test = model.predict(X_test)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    mae_test = mean_absolute_error(Y_test, Y_pred_test)
    
    print("MSE_test: ", mse_test)
    print("MAE_test: ", mae_test)
    
    return model

In [97]:
def paitplot(df2):
    res = df2[~df2.duplicated('City')]['City']
    res = res.reset_index(drop=True)
    print(res)

    for i in range(len(res)):
        print(df2[df2['City'] == res[i]].shape)
        plot = sns.pairplot(df2[df2['City'] == res[i]])
        plot.savefig(f"output/{res[i]}.png") 